In [ ]:
# default_exp recentFeed

# recentFeed

> Parse the recent filings feed.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export

import re
import xml.etree.cElementTree as cElTree

from secscan import utils